In [1]:
def enter_block():
    """Function that takes input of a hexadecimal number"""
    while True:
        textblock = input("Enter a 16-bit hexadecimal number:")
        try:
            decimal = int(textblock, 16)
            if 0 <= decimal <= 0xFFFF:
                #calculating the binary equivalent
                binaryeq = bin(decimal)
                return textblock
            else:
                print("Input is not within the 16-bit range.")
        except ValueError:
            print("Input is not a hexadecimal number.")

In [2]:

def convertmatrix(textblock):
    
    while len(textblock) % 2 != 0:
        textblock += '0'

    
    matrix = [['0' for _ in range(2)] for _ in range(2)]  #initialising the matrix with 0

    # Populate the matrix column by column
    for i in range(2):
        for j in range(2):
            matrix[j][i] = textblock[i * 2 + j]   # i * 2 + j writes the text block column wise to the matrix

    return matrix


In [3]:
stable = {
    '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
    '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
    '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
    '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
}

reverse_stable = {
    '1010': '0000', '0000': '0001', '1001': '0010', '1110': '0011',
    '0110': '0100', '0011': '0101', '1111': '0110', '0101': '0111',
    '0001': '1000', '1101': '1001', '1100': '1010', '0111': '1011',
    '1011': '1100', '0100': '1101', '0010': '1110', '1000': '1111'
}


In [4]:
def subnibble(hex_string, reverse):
    result = ''
    if reverse == False:
        s_box = stable 
    else:
        #s_box = {v: k for k, v in stable.items()}
        s_box = reverse_stable

    binaryeq = ''.join(bin(int(hex_digit, 16))[2:].zfill(4) for hex_digit in hex_string)
    for i in range(0, len(binaryeq), 4):
        nibble = binaryeq[i:i+4]
        subnibble = s_box[nibble]
        result += subnibble
    return hex(int(result, 2))[2:].zfill(4)


In [5]:

def shift_row(matrix):
    """Performs shift row opeartion on the matrix"""
    if len(matrix) != 2 or len(matrix[0]) != 2 or len(matrix[1]) != 2:
        raise ValueError("Input matrix must be a 2x2 matrix.")
    
    matrix[0] = matrix[0][-1:] + matrix[0][:-1]
    #print("Shiftrow: " + matrix[0][0] + matrix[1][0] + matrix[0][1] + matrix[1][1])
    return matrix



In [6]:
def finite_field_multiply(a, b):
    a_int = int(a)  # Convert 'a' to an integer (assuming it's already in binary)
    b_int = int(b, 16)  # Convert 'b' from a hexadecimal
    m = 0
    
    while (b_int) > 0:
        if b_int & 1:  
            m = m ^ a_int  
        
        if a_int & 0x8:  
            a_int = (a_int << 1) ^ int('0x13', 16)  
        else:
            a_int <<= 1  
        
        b_int >>= 1 
    return m

In [7]:
mat = [[0,0],
       [0,0]]

constmat = [[1, 4],
            [4, 1]]

inversemat = [[9,2],
              [2,9]]

In [8]:
def mix_columns(cmat, matrix):
    mat[0][0] = finite_field_multiply(cmat[0][0], matrix[0][0]) ^ finite_field_multiply(cmat[0][1], matrix[1][0])
    mat[1][0] = finite_field_multiply(cmat[1][0], matrix[0][0]) ^ finite_field_multiply(cmat[1][1], matrix[1][0])
    mat[0][1] = finite_field_multiply(cmat[0][0], matrix[0][1]) ^ finite_field_multiply(cmat[0][1], matrix[1][1])
    mat[1][1] = finite_field_multiply(cmat[1][0], matrix[0][1]) ^ finite_field_multiply(cmat[1][1], matrix[1][1])
    
    mat[0][0] = hex(mat[0][0])[2:]
    mat[1][0] = hex(mat[1][0])[2:]
    mat[0][1] = hex(mat[0][1])[2:]
    mat[1][1] = hex(mat[1][1])[2:]
    
    val = mat[0][0] + mat[1][0] + mat[0][1] + mat[1][1]
    return mat




In [9]:
print('MixColumns:' , mix_columns(constmat, convertmatrix(enter_block())))

MixColumns: [['9', '9'], ['2', '7']]


In [10]:
def get_hexadecimal_key():
    while True:
        user_input = input("Enter a 16-bit hexadecimal key: ").strip()  
        if len(user_input) == 4 and all(c in '0123456789abcdefABCDEF' for c in user_input):
            return user_input  

        print("Invalid input. Please enter a 16-bit hexadecimal key")



In [9]:
keymat = convertmatrix(get_hexadecimal_key())

In [11]:
def generate_roundkeys(keymat, reverse):
    w4 = int(keymat[0][0], 16) ^ int(subnibble(keymat[1][1], reverse),16) ^ int('e', 16)
    w5 = int(keymat[1][0], 16) ^ w4
    w6 = int(keymat[0][1], 16) ^ w5
    w7 = int(keymat[1][1], 16) ^ w6

    hw4 = hex(w4)[2:]
    hw5 = hex(w5)[2:]
    hw6 = hex(w6)[2:]
    hw7 = hex(w7)[2:]

    roundkey1 = hw4 + hw5 + hw6 + hw7

    w8 = w4 ^ int(subnibble(hw7 , reverse), 16) ^ int('a', 16)
    w9 = w5 ^ w8
    w10 = w6 ^ w9
    w11 = w7 ^ w10
    
    hw8 = hex(w8)[2:]
    hw9 = hex(w9)[2:]
    hw10 = hex(w10)[2:]
    hw11 = hex(w11)[2:]
    roundkey2 = hw8 + hw9 + hw10 + hw11

    return roundkey1 , roundkey2 




In [17]:
#Block of code that tests each of the functions used above
textblock = enter_block()
print("Deliverable 1")
print("TextBlock:", textblock)
# key = get_hexadecimal_key()
print('SubNibbles(' + textblock + '): ' + subnibble(textblock, False))
sr = shift_row(convertmatrix(textblock))
print("Shiftrow("+ textblock+'): ' + sr[0][0] + sr[1][0] + sr[0][1] + sr[1][1])
mc = mix_columns(constmat, convertmatrix(textblock))
print("Mix Columns(" + textblock + '): ' + mc[0][0] + mc[1][0] + mc[0][1] + mc[1][1])
key = get_hexadecimal_key()
r1 , r2 = generate_roundkeys(convertmatrix(key), False)
print("Generated Round Keys(" + key + '): (' + r1 + ' , ' + r2 + ')')

Deliverable 1
TextBlock: 903b
SubNibbles(903b): dae7
Shiftrow(903b): 309b
Mix Columns(903b): 9297
Generated Round Keys(02cc): (57b7 , ad61)


In [13]:
def decryption(textblock, key):
    """Function that takes the cipher text and the key as input and returns the plain text"""
    shifted_row = shift_row(convertmatrix(textblock))
    
    k1, k2 = generate_roundkeys(convertmatrix(key), False)
  
    shiftval = shifted_row[0][0] + shifted_row[1][0] + shifted_row[0][1] + shifted_row[1][1]
    
    xorval =  int(k2, 16) ^ int(shiftval, 16) 
    xorval =hex(xorval)[2:].zfill(4)
    snval = subnibble(xorval, True)
    
    shifted_row = shift_row(convertmatrix(snval))
  
    mixmat = mix_columns(inversemat , shifted_row)
   
    mixstr = mixmat[0][0] + mixmat[1][0] + mixmat[0][1] + mixmat[1][1]
  
    xorval = int(k1, 16) ^ int(mixstr, 16)
   
    snval = subnibble(hex(xorval)[2:], True)
   
    return snval
 

In [18]:
#Block of code that tests the decryption functions used above
textblock = enter_block()
print("Deliverable 2")
print("Cipher Text Block: " + textblock)
key = get_hexadecimal_key()
print("Key Entered: " + key)
decrypted_block = decryption(textblock , key)
print("Decrypted Block: " + decrypted_block)

Deliverable 2
Cipher Text Block: f3d7
Key Entered: 40ee
Decrypted Block: e282


In [15]:
def encryption(textblock, key):
    """Function that takes the plain text and the key as input and returns the cipher text"""
    snval = subnibble(textblock , False)
    k1, k2 = generate_roundkeys(convertmatrix(key) , False)
    xorval = int(snval, 16) ^ int(k1, 16)
    mixmat = mix_columns(constmat, convertmatrix(hex(xorval)[2:]))
    shifted_row = shift_row(mixmat)
    srval = shifted_row[0][0] + shifted_row[1][0] + shifted_row[0][1] + shifted_row[1][1]
    sn = subnibble(srval, False)
    xorval = int(sn, 16) ^ int(k2, 16)
    shifted_row = shift_row(convertmatrix(hex(xorval)[2:]))
    encrypted_value = shifted_row[0][0] + shifted_row[1][0] + shifted_row[0][1] + shifted_row[1][1]
    print(encrypted_value)
encryption('2e00' , '149c')

731f


In [19]:


def decrypt_text(ciphertext, key):
    """Function that takes the cipher text and the key as input and returns the plain text"""
    plaintext = []
    strtext = ''
    print(len(ciphertext))
    for i in range(0, len(ciphertext)-1, 4):
        encrypted_block = ciphertext[i:i+4]
        decrypted_block = decryption(encrypted_block, key)

        for j in range(0, len(decrypted_block), 2):
            char = chr(int(decrypted_block[j:j+2], 16))
            strtext += char
            plaintext.append(char)
    print("Decrypted Result:")
    print("------------------------------------")
    print(strtext)
    print("-----------------------------------------------")
    nullremovedtext = ''.join(plaintext).rstrip('\x00')
    return ''.join(nullremovedtext)
    

with open('secret.txt', 'r') as file:
    cipher = file.read().replace(" ", "")

key = get_hexadecimal_key()

plaintext = decrypt_text(cipher, key)


with open('plain.txt', 'w') as file:
    file.write(plaintext)




116
Decrypted Result:
------------------------------------
Gentlemen, you can't fight in here. This is the war room. 
-----------------------------------------------
